In [1]:
class Opion():
    
    def __init__(self):
        
        self.dataroot= r'/home/jara/DeepInPainting_3/paris_original_test' #image dataroot
        self.maskroot= r'/home/jara/DeepInPainting_3/mask/test_mask'
        self.batchSize= 1   # Need to be set to 1
        self.fineSize=256 # image size
        self.input_nc=3  # input channel size for first stage
        self.input_nc_g=6 # input channel size for second stage
        self.output_nc=3# output channel size
        self.ngf=64 # inner channel
        self.ndf=64# inner channel
        self.which_model_netD='basic' # patch discriminator
        
        self.which_model_netF='feature'# feature patch discriminator
        self.which_model_netG='unet_csa'# seconde stage network
        self.which_model_netP='unet_256'# first stage network
        self.triple_weight=1
        self.name='CSA_inpainting'
        self.n_layers_D='3' # network depth
        self.gpu_ids=[0]
        self.model='csa_net'
        self.checkpoints_dir=r'/home/jara/DeepInPainting_3/checkpoints' #
        self.norm='instance'
        self.fixed_mask=1
        self.use_dropout=False
        self.init_type='normal'
        self.mask_type='random'
        self.lambda_A=100
        self.threshold=5/16.0
        self.stride=1
        self.shift_sz=1 # size of feature patch
        self.mask_thred=1
        self.bottleneck=512
        self.gp_lambda=10.0
        self.ncritic=5
        self.constrain='MSE'
        self.strength=1
        self.init_gain=0.02
        self.cosis=1
        self.gan_type='lsgan'
        self.gan_weight=0.2
        self.overlap=4
        self.skip=0
        self.display_freq=1000
        self.print_freq=50
        self.save_latest_freq=5000
        self.save_epoch_freq=2
        self.continue_train=False
        self.epoch_count=1
        self.phase='train'
        self.which_epoch=46
        self.niter=20
        self.niter_decay=100
        self.beta1=0.5
        self.lr=0.0002
        self.lr_policy='lambda'
        self.lr_decay_iters=50
        self.isTrain=False

    

In [2]:
import time
from util.data_load import Data_load
from models.models import create_model
import torch
import os
import torchvision
from torch.utils import data
import torchvision.transforms as transforms
from IQA_pytorch import SSIM
import numpy as np
opt = Opion()
transform_mask = transforms.Compose(
    [transforms.Resize((opt.fineSize,opt.fineSize)),
     transforms.ToTensor(),
    ])
transform = transforms.Compose(
    [
     transforms.Resize((opt.fineSize,opt.fineSize)),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)])
transform_ref = transforms.Compose(
    [transforms.RandomResizedCrop((opt.fineSize,opt.fineSize),scale=(0.8,1.0),ratio=(1,1)),
     transforms.ColorJitter(brightness=0.1,contrast=0.1,saturation=0.1,hue=0.1),
     transforms.ToTensor(),
     transforms.Normalize(mean=[0.5] * 3, std=[0.5] * 3)])

dataset_test = Data_load(opt.dataroot, opt.maskroot,opt.dataroot, transform, transform_mask,transform_ref)
iterator_test = (data.DataLoader(dataset_test, batch_size=opt.batchSize,shuffle=True))
print(len(dataset_test))
model = create_model(opt)
total_steps = 0


2400
csa_net
initialize network with normal


/home/jara/DeepInPainting_3/models/networks.py:54: UserWarning: nn.init.normal is now deprecated in favor of nn.init.normal_.
  init.normal(m.weight.data, 0.0, gain)
/home/jara/DeepInPainting_3/models/networks.py:64: UserWarning: nn.init.constant is now deprecated in favor of nn.init.constant_.
  init.constant(m.bias.data, 0.0)


initialize network with normal
Loading pre-trained network!
model [CSAModel] was created


In [3]:
load_epoch=46
model.load(load_epoch)

In [4]:

save_dir = r'/home/jara/DeepInPainting_3/result'
if os.path.exists(save_dir) is False:
    os.makedirs(save_dir)

epoch=1
i=0
for image, mask,ref in (iterator_test):
    iter_start_time = time.time()
    image=image.cuda()
    mask=mask.cuda()
    mask=mask[0][0]
    mask=torch.unsqueeze(mask,0)
    mask=torch.unsqueeze(mask,1)
    mask=mask.bool()

    model.set_input(image,mask,ref) # it not only sets the input data with mask, but also sets the latent mask.
    model.set_ref_latent()
    model.set_gt_latent()
    model.test()
    
    real_A,ref_B,fake_B,fake_P,real_B=model.get_current_visuals()
    pic = (torch.cat([real_A, real_B,fake_P,fake_B], dim=0) + 1) / 2.0
    
    torchvision.utils.save_image(pic, '%s/Epoch_(%d)_(%dof%d).jpg' % (
    save_dir, epoch, total_steps + 1, len(dataset_test)), nrow=2)
    
    psnr=0
    mse=0
    if type(real_B)==torch.Tensor:
        mse=torch.mean((real_B-fake_B)**2)
    else:
        mse=np.mean((real_B-fake_B)**2)
    
    if mse==0:
        psnr=100
    else:
        psnr=10*torch.log10((2**2)/mse)
    errors=model.get_error()
    
    mod=SSIM(channels=3)
    score=mod(real_B,fake_B,as_loss=False)
    print('PSNR : %f, SSIM : %f' %
         (psnr,score))
    epoch=epoch+1


PSNR : 18.598665, SSIM : 0.617952
PSNR : 32.201630, SSIM : 0.972471
PSNR : 20.955736, SSIM : 0.574273
PSNR : 34.251842, SSIM : 0.966834
PSNR : 26.054447, SSIM : 0.777909
PSNR : 29.522341, SSIM : 0.866079
PSNR : 23.568851, SSIM : 0.662409
PSNR : 28.063644, SSIM : 0.937558
PSNR : 27.253382, SSIM : 0.845498
PSNR : 25.969440, SSIM : 0.854758
PSNR : 37.702785, SSIM : 0.959402
PSNR : 24.561367, SSIM : 0.691677
PSNR : 29.304138, SSIM : 0.881763
PSNR : 23.661583, SSIM : 0.761094


KeyboardInterrupt: 